<a href="https://colab.research.google.com/github/cy643/generative_ai/blob/main/%E6%9C%9F%E6%9C%AB%E5%B0%88%E6%A1%88/%E8%B3%87%E6%96%99%E7%88%AC%E8%9F%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 從故宮開源網站爬蟲資料


In [ ]:
!pip install requests beautifulsoup4 pandas sentence-transformers tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import os
import re


In [ ]:
!pip install selenium
!apt-get update # 安裝 Chrome Driver
!apt install chromium-chromedriver


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 24.8 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,703 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,953 kB]
Hit:11 https://ppa.launchpadcontent.

## 利用互動式套件讀取網頁html source code標籤進行篩選與翻頁

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

# 設定 Chrome 為 headless 模式
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")


In [ ]:
driver = webdriver.Chrome(options=chrome_options)
BASE_URL = "https://digitalarchive.npm.gov.tw/opendata"
driver.get(BASE_URL)

### 抓取說明頁面中的文字

In [ ]:

def extract_artifact_details(card_element):
    try:
        # 點擊卡片
        card_element.click()
        time.sleep(1)  # 等動畫結束

        # 等待詳細資料出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.details-1"))
        )

        # 擷取詳細 HTML
        detail_soup = BeautifulSoup(driver.page_source, "html.parser")
        detail_div = detail_soup.select_one("div.details-1")
        paragraphs = detail_div.find_all("p")

        detail_info = [p.text.strip() for p in paragraphs]
        print("✅ 詳細資訊：", detail_info)

        # 回到主頁（點返回箭頭）
        back_btn = driver.find_element(By.CLASS_NAME, "go-back")  # 根據故宮的返回按鈕 class 名稱
        back_btn.click()
        time.sleep(1)
        return detail_info

    except Exception as e:
        print("❌ 抓取失敗：", e)
        return []


### 跳轉至下一頁搜索頁面

In [ ]:
def click_next_page(previous_first_title=None):
    try:
        next_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@data-bs-original-title='下一頁']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("➡️ 點擊下一頁成功，等待新內容")

        # 等第一筆卡片變更
        if previous_first_title:
            WebDriverWait(driver, 10).until(
                lambda d: d.find_elements(By.CSS_SELECTOR, "a.card") and
                          d.find_elements(By.CSS_SELECTOR, "a.card")[0]
                          .find_element(By.CLASS_NAME, "card-title").text.strip() != previous_first_title
            )
        time.sleep(1)
        return True
    except Exception as e:
        print("🚫 下一頁點擊失敗：", e)
        return False


### 從說明頁跳轉回搜索頁

In [ ]:
def safe_back_to_main():
    try:
        return_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, "ReturnToPage"))
        )
        driver.execute_script("arguments[0].click();", return_button)
        time.sleep(1)
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.card"))
        )
        # print("🔙 成功點擊『返回列表』")
    except Exception as e:
        print("⚠️ 點擊返回列表失敗，改用 driver.get：", e)
        driver.get(BASE_URL)
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.card"))
        )


### 抓取當前搜索頁面中的所有搜尋結果，並將說明頁面的圖像與說明文字分類與儲存
只有說明文字完整(四條細節說明)的搜尋結果會被儲存

In [ ]:
def crawl_npm_artifacts(page):

    all_data = []

    # for page in range(pages):

    print(f"📄 第 {page} 頁")

    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.card"))
    )
    cards = driver.find_elements(By.CSS_SELECTOR, "a.card")
    print(f"  ⬇️ 抓到 {len(cards)} 筆資料")

    i = 0
    while i < len(cards):
        try:
            cards = driver.find_elements(By.CSS_SELECTOR, "a.card")
            if i >= len(cards):
                print(f"⚠️ 第 {i+1} 筆超出卡片數量，略過")
                break

            card = cards[i]
            title = card.find_element(By.CLASS_NAME, "card-title").text.strip()
            img_tag = card.find_element(By.TAG_NAME, "img")
            img_url = "https://digitalarchive.npm.gov.tw" + img_tag.get_attribute("src")

            driver.execute_script("arguments[0].click();", card)
            time.sleep(1)

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.details-1"))
            )

            detail_soup = BeautifulSoup(driver.page_source, "html.parser")
            detail_div = detail_soup.select_one("div.details-1")
            detail_paragraphs = detail_div.find_all("p") if detail_div else []
            detail_texts = [p.text.strip() for p in detail_paragraphs]

            # 分類處理 detail_texts
            if len(detail_texts) == 4:
                print(f"✅ [{i+1}] {title} - 取得 {len(detail_texts)} 筆細節")
                cat = detail_texts[0]
                era = detail_texts[1]
                size_and_desc = detail_texts[2] + " " + detail_texts[3]
                # 加入結果
                all_data.append({
                    "title": title,
                    "image_url": img_url,
                    "category": cat,
                    "era": era,
                    "size_and_description": size_and_desc
                })


            # 回主頁
            safe_back_to_main()
            time.sleep(1)
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.card"))
            )

        except Exception as e:
            print(f"❌ 第 {i+1} 筆處理失敗：", e)
            title = "錯誤資料"
            img_url = None
            detail_texts = []

            # 強制刷新回主頁以避免異常
            print("🔄 嘗試重新整理主頁")
            safe_back_to_main()
            time.sleep(2)
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.card"))
            )

        i += 1

    first_card_title = ""
    try:
        first_card_title = driver.find_elements(By.CSS_SELECTOR, "a.card")[0]\
            .find_element(By.CLASS_NAME, "card-title").text.strip()
    except:
        print("⚠️ 無法取得當前第一筆標題")

        # # 翻頁
        # if not click_next_page(previous_first_title=first_card_title):
        #     break

    return pd.DataFrame(all_data), first_card_title


In [ ]:
from selenium.webdriver.support import expected_conditions as EC

def wait_for_nonempty_text(by, locator, timeout=10):
    def _predicate(driver):
        try:
            el = driver.find_element(by, locator)
            text = el.text.strip()
            return text if text and text.isdigit() else False
        except:
            return False
    return WebDriverWait(driver, timeout).until(_predicate)


### 取得搜索頁面的總頁數，並且根據設定的間距直接跳轉到指定頁數

In [ ]:
def get_total_page_count():
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "total-pageCount"))
        )
        for _ in range(10):
            text = driver.execute_script("""
                const el = document.getElementById('total-pageCount');
                return el ? el.innerText.trim() : '';
            """)
            print("🧪 JS讀到頁數：", repr(text))
            if text.isdigit():
                return int(text)
            time.sleep(0.5)
        raise ValueError("頁數仍為空或非數字")
    except Exception as e:
        print(f"⚠️ 無法取得總頁數（JS版）：{e}")
        return 0


In [ ]:
def jump_to_page(page_num, previous_first_title):
    try:
        driver.execute_script(f"ChangePageIndex({page_num})")
        if previous_first_title:
            WebDriverWait(driver, 10).until(
                lambda d: d.find_elements(By.CSS_SELECTOR, "a.card") and
                          d.find_elements(By.CSS_SELECTOR, "a.card")[0]
                          .find_element(By.CLASS_NAME, "card-title").text.strip() != previous_first_title
            )
        time.sleep(1)
        print(f"➡️ 已跳轉至第 {page_num} 頁")
        return True
    except Exception as e:
        print(f"❌ 跳轉第 {page_num} 頁失敗：{e}")
        return False

### 每隔n頁抓取一次搜尋結果

In [ ]:
def crawl_every_nth_page(n=10, max_pages=None):
    all_data = []
    total_pages = get_total_page_count()

    if max_pages:
        total_pages = min(total_pages, max_pages)
    print(f"最大頁數:{total_pages}")
    previous_first_title = ""
    for page in range(1, total_pages + 1, n):
        print(f"\n📘 嘗試跳轉第 {page} 頁")
        success = jump_to_page(page, previous_first_title)
        if not success:
            continue

        # 等畫面穩定
        time.sleep(1)

        # 抓該頁的資料
        df, previous_first_title = crawl_npm_artifacts(page)  # 只抓這一頁
        all_data.extend(df.to_dict("records"))


    return pd.DataFrame(all_data)


## 執行
我已知全部搜索頁面約有7000頁，設定每隔50頁抓取搜索結果，直到第7000頁
最終將搜索結果根據分類存成csv檔，需要下載到本機端

In [ ]:
# df = crawl_npm_artifacts(pages=1)
# df.to_csv("npm_artifacts_with_details.csv", index=False)
# df.head()


In [ ]:
df = crawl_every_nth_page(n=50,max_pages=7000)
df.to_csv("npm_artifacts_with_details.csv", index=False)
df.head()

🧪 JS讀到頁數： '7164'
最大頁數:7000

📘 嘗試跳轉第 1 頁
➡️ 已跳轉至第 1 頁
📄 第 1 頁
  ⬇️ 抓到 15 筆資料
✅ [1] 「臨池真賞」墨 - 取得 4 筆細節
✅ [5] 「大壑五山」圓墨 - 取得 4 筆細節
✅ [6] 「六龍御天頌」圓墨 - 取得 4 筆細節
✅ [13] 御題《萬春集慶》冊〈萬壑春雲〉詩綠墨 - 取得 4 筆細節
✅ [15] 御題《萬春集慶》冊〈萬派春濤〉詩藍墨 - 取得 4 筆細節

📘 嘗試跳轉第 51 頁
➡️ 已跳轉至第 51 頁
📄 第 51 頁
  ⬇️ 抓到 15 筆資料
✅ [1] 澄泥玉堂硯 - 取得 4 筆細節
✅ [2] 仿澄泥虎伏硯 - 取得 4 筆細節
✅ [3] 端石海天浴日硯 - 取得 4 筆細節
✅ [9] 暖硯 - 取得 4 筆細節
✅ [10] 澄泥鳳鏡硯 - 取得 4 筆細節
✅ [13] 澄泥龍珠硯 - 取得 4 筆細節
✅ [14] 澄泥蟠螭硯 - 取得 4 筆細節

📘 嘗試跳轉第 101 頁
➡️ 已跳轉至第 101 頁
📄 第 101 頁
  ⬇️ 抓到 15 筆資料
✅ [1] 御詠花卉詩「翠雀」藍墨 - 取得 4 筆細節
✅ [2] 御詠花卉詩「蘭」綠墨 - 取得 4 筆細節
✅ [3] 御詠花卉詩「珍珠蘭」綠墨 - 取得 4 筆細節
✅ [4] 御詠花卉詩「石竹」綠墨 - 取得 4 筆細節
✅ [5] 御詠花卉詩「鳳仙」綠墨 - 取得 4 筆細節
✅ [6] 御詠花卉詩「梅」綠墨 - 取得 4 筆細節
✅ [7] 御詠花卉詩「瑞香」白墨 - 取得 4 筆細節
✅ [8] 御詠花卉詩「水仙」白墨 - 取得 4 筆細節
✅ [9] 御詠花卉詩「梨花」白墨 - 取得 4 筆細節
✅ [10] 御詠花卉詩「玉蘭」白墨 - 取得 4 筆細節
✅ [11] 御詠花卉詩「繡毬花」白墨 - 取得 4 筆細節
✅ [12] 御詠花卉詩「紫薇」硃墨 - 取得 4 筆細節
✅ [14] 御詠花卉詩「荷」硃墨 - 取得 4 筆細節
✅ [15] 御詠花卉「錦葵」詩硃墨 - 取得 4 筆細節

📘 嘗試跳轉第 151 頁
➡️ 已跳轉至第 151 頁
📄 第 151 頁
  ⬇️ 抓到 15 筆資料
✅ [1] 玉人 - 取得 4 筆細節
✅ [2] 玉海棠式盆托 - 取

,title,image_url,category,era,size_and_description
0,「臨池真賞」墨,https://digitalarchive.npm.gov.twhttps://digit...,文具,清 康熙五十五年,高23.0公分 徑長4.6公分 寬4.2公分 八方柱狀墨，質堅緻。一面有陰文填金篆字︰「臨池...
1,「大壑五山」圓墨,https://digitalarchive.npm.gov.twhttps://digit...,文具,明,高2.0公分 徑16.3公分 圓形墨，邊框起棱，一面模印明代文人焦竑所書〈大壑五山〉銘文，並...
2,「六龍御天頌」圓墨,https://digitalarchive.npm.gov.twhttps://digit...,文具,明,徑16.0公分 圓形墨，邊框起棱，一面模印程君房所作之〈六龍御天頌〉。另面則模印六龍穿梭於雲...
3,御題《萬春集慶》冊〈萬壑春雲〉詩綠墨,https://digitalarchive.npm.gov.twhttps://digit...,文具,清 嘉慶十四年,長11.36公分 寬5.37公分 高1.82公分 長方形綠色顏料墨，細膩鮮豔。雙面起邊框，皆...
4,御題《萬春集慶》冊〈萬派春濤〉詩藍墨,https://digitalarchive.npm.gov.twhttps://digit...,文具,清 嘉慶十四年,長11.57公分 寬5.49公分 高1.79公分 長方形藍色顏料墨，細膩鮮豔。雙面起邊框，皆...
